## Bài toán
 - Dự đoán khả năng tiến triển của bệnh tiểu đường thông qua các chỉ số sinh lý của cơ thể.


## Thông tin dữ liệu:

- Số lượng mẫu: 442 (thông tin từ 442 bệnh nhân)
- Thông tin ccs thuộc tính (10 cột đầu tiên):
  - Age(tuổi)
  - Sex (giới tính) 
  - Body mass index (chỉ số khối cơ thể)
  - Average blood pressure(huyết ap trung bình)
  - S1, S2, S3, S4, S5, S6 (sáu phép đo huyết thanh khác)
- Mục tiêu: Cột 11, chỉ số đánh giá mức độ tiến triển của bệnh sau 1 năm điều trị

**! Chú ý: Các thuộc tính trong dữ liệu đã được chuẩn hóa**

Xem thêm thông tin về nguồn dữ liệu tại: (https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html)


## Hướng giải quyết

Giả sử rằng khả năng tiến triển của bệnh tiểu đường (ký hiệu: `y`) là đại lượng phụ thuộc tuyến tính vào các thông tin sinh lý của bệnh nhân như các thuộc tính đã mô tả ở trên (tuổi, giới tính, chỉ số khối, ... - ký hiệu: $x_1, x_2, .. x_n$) : 

$y = w_0 + w_1x_1 + w_1x_2 + ... + w_nx_n$

Mục tiêu: Tìm được bộ trọng số $[w_0, w_1, w_2... w_n]$ biểu diễn mối quan hệ này. 

## Thư viện sử dụng 

- matplotlib: phục vụ vẽ các đồ thị 
- numpy: tính toán các phép biến đổi trên ma trận / vector 
- math: thực hiện một số hàm tính toán đơn giản
- pandas: Xử lý dữ liệu bảng hai chiều 
- scikit-learn: (sklearn) thư viện hỗ trợ xây dựng các mô hình học máy, các hàm huấn luyện và đánh giá 


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import math 
import seaborn as sns

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score


## 1. Đọc dữ liệu huấn luyện

Dữ liệu huấn luyện về bệnh tiểu đường được lưu dưới dạng file .pkl với hai thuộc tính:
- data: ma trận 2 chiều (400x10) - các thông tin bệnh nhân được chuẩn hoá về dạng số thực
- target: mảng các số thực (400,) - chỉ số tiến triển của bệnh tiểu đường

In [ ]:
with open('diabetes_train.pkl', 'rb') as f:
    diabetes_train = pickle.load(f)
print("Số chiều input: ", diabetes_train['data'].shape)
print("Số chiều target y tương ứng: ", diabetes_train['target'].shape)
print()

print("2 mẫu dữ liệu đầu tiên:")
print("input: ", diabetes_train['data'][:2])
print("target: ",diabetes_train['target'][:2])

## 2. Xây dựng mô hình hồi quy sử dụng Sklearn

Thử nghiệm xây dựng và huấn luyện mô hình hồi quy tuyến tính (Linear Regression)
- [Linear Regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) `linear_model.LinearRegression()`

In [ ]:
# Xây dựng mô hình sử dụng thư viện scikit-learn
regr = linear_model.LinearRegression()


## 3. Huấn luyện mô hình

Sử dụng dữ liệu đã được chia ở bước trước đó để thực hiện huấn luyện mô hình 

=> Tìm được bộ trọng số $[w_0, w_1, ... w_n]$

In [ ]:
# huấn luyện mô hình
regr.fit(diabetes_train['data'], diabetes_train['target'])
print("[w1, ... w_n] = ", regr.coef_)
print("w0 = ", regr.intercept_)

## 4. Dự đoán các mẫu dữ liệu trong tập test 

# 4.1 Phán đoán

In [ ]:
# đọc dữ liệu test
# dữ liệu test có cấu trúc giống dữ liệu huấn luyện nhưng số lượng mẫu chỉ là 42
with open('diabetes_test.pkl', 'rb') as f:
    diabetes_test = pickle.load(f)

# Thực hiện phán đoán cho dữ liệu mới
diabetes_y_pred = regr.predict(diabetes_test['data'])

## 4.2 Kiểm tra chất lượng phán đoán

In [ ]:
pd.DataFrame(data=np.array([diabetes_test['target'], diabetes_y_pred,
                            abs(diabetes_test['target'] - diabetes_y_pred)]).T,
             columns=["Thực tế", "Dự đoán", "Lệch"])

Sử dụng độ đo RMSE tính căn bậc 2 của trung bình bình phương lỗi. 
> $\text{RMSE}(y, \hat{y}) = \sqrt{\frac{1}{m} \sum\limits_{i=1}^{m} (y_i - \hat{y}_i)^2}$

- Lỗi càng nhỏ càng thể hiện mô hình có khả năng học và dự đoán hiệu quả

In [ ]:
rmse = math.sqrt(mean_squared_error(diabetes_test['target'], diabetes_y_pred))
print(f'RMSE = {rmse}')

Phân phối các dự đoán đầu ra của mô hình

In [ ]:
sns.displot(diabetes_y_pred, )

Kết quả dự đoán của mô hình tuân theo phân phối chuẩn trong đó:

- Khoảng giá trị giao động từ 0 đến khoảng 260
- Phần lớn các dữ đoán nằm gần 150